In [8]:
import openrouteservice
import plotly.graph_objects as go
import requests
from geopy.geocoders import Nominatim
from geopy.distance import geodesic


# OpenRouteService API
openrouteservice_api_key = "YOUR KEY"

# OpenWeatherMap API
openweathermap_api_key = "YOUR KEY"

# OpenRouteService call
ors_client = openrouteservice.Client(key=openrouteservice_api_key)

# Geopy geocoding service
geolocator = Nominatim(user_agent="geoapiExercises")

def get_coordinates_from_location(location):
    try:
        result = ors_client.pelias_search(location)
        coordinates = result['features'][0]['geometry']['coordinates']
        return coordinates
    except Exception as e:
        print(f"Error: {e}")
        return None

def get_osm_route(start_location, end_location):
    if start_location and end_location:
        try:
            route = ors_client.directions(coordinates=[start_location, end_location], profile='driving-car', format='geojson', validate=True)
            return route
        except openrouteservice.exceptions.ApiError as e:
            print(f"Failed to receive route information. Error code: {e.status_code}, Error message: {e.message}")
            return None
    else:
        print("Coordinates could not be received.")
        return None

def get_weather(api_key, lat, lon):
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {"lat": lat, "lon": lon, "appid": api_key, "units": "metric"}

    try:
        response = requests.get(base_url, params=params)
        data = response.json()

        if response.status_code == 200:
            weather_description = data['weather'][0]['description']
            temperature = data['main']['temp']
            return f"{weather_description} ({temperature}°C)"
        else:
            return f"Failed to receive weather information. Error code: {response.status_code}"
    except Exception as e:
        return f"Error: {e}"

# User input
start_location_input = input("Enter the departure city or address: ")
end_location_input = input("Enter the destination city or address: ")

# Geocode input to coordinates
start_location = get_coordinates_from_location(start_location_input)
end_location = get_coordinates_from_location(end_location_input)

# Get route information
route = get_osm_route(start_location, end_location)

if route:
    print("Route information:")
    for step in route['features'][0]['properties']['segments'][0]['steps']:
        print(step['instruction'])

    weather_info = []
    last_point = None
    for i, point in enumerate(route['features'][0]['geometry']['coordinates']):
        lat, lon = point[1], point[0]

        if last_point is None or geodesic(last_point, (lat, lon)).km >= 25:
            weather_info.append(get_weather(openweathermap_api_key, lat, lon))
            last_point = (lat, lon)
        else:
            weather_info.append(None)

    # Plotly map
    fig = go.Figure()

    # Route Line
    fig.add_trace(go.Scattermapbox(
        mode="lines",
        lon=[point[0] for point in route['features'][0]['geometry']['coordinates']],
        lat=[point[1] for point in route['features'][0]['geometry']['coordinates']],
        marker={'size': 10}
    ))

    # Weather infos and emojis
    for i, point in enumerate(route['features'][0]['geometry']['coordinates']):
        lat, lon = point[1], point[0]

        if weather_info[i] is not None:
            emoji = '☁️' if 'cloud' in weather_info[i].lower() else '☔' if 'rain' in weather_info[i].lower() else '🌬️'
            text = f"{emoji} {weather_info[i]}"
            fig.add_trace(go.Scattermapbox(
                mode="markers+text",
                lon=[lon],
                lat=[lat],
                marker={'size': 12, 'color': 'red'},
                text=[text],
                hoverinfo='text'
            ))

    fig.update_layout(
        mapbox={
            'style': "open-street-map",
            'center': {'lon': route['features'][0]['geometry']['coordinates'][0][0], 'lat': route['features'][0]['geometry']['coordinates'][0][1]},
            'zoom': 10
        },
        margin={"r": 0, "t": 0, "l": 0, "b": 0}
    )

    fig.show()

else:
    print("Failed to receive route information.")


Enter the departure city or address: Żwirki i Wigury 1, 02-143 Warszawa
Enter the destination city or address: Przemysłowa 2, 66-400 Gorzów Wielkopolski
Route information:
Head northwest
Turn left
Turn left onto Żwirki i Wigury
Keep right
Enter the roundabout and take the 2nd exit onto Wiktora Narkiewicza
Turn left onto Stefana Bastyra
Turn right onto Wandy Modlibowskiej
Keep right onto Wandy Modlibowskiej
Keep right
Turn right
Turn left
Keep right
Keep left
Keep right
Turn left onto Marcina Kasprzaka, 22
Turn right onto Marcina Kasprzaka
Turn slight left
Turn left onto Koniawska
Enter the roundabout and take the 3rd exit
Turn right onto Przemysłowa
Arrive at Przemysłowa, on the right
